In [ ]:
import importlib
from pathlib import Path

import pandas as pd
from rich.console import Console

c = Console(width=60)

GDRIVE = Path("~/Google Drive/Shared drives/Solutions").expanduser()

# Quick Start

In [ ]:
from cuery import Prompt, Response, Task, set_env

set_env(apify_secrets=False)


# Define response model
class Entity(Response):
    name: str
    type: str


class NamedEntities(Response):
    entities: list[Entity]


context = pd.DataFrame(
    {
        "text": [
            "Apple is headquartered in Cupertino, California.",
            "Barack Obama was the 44th President of the United States.",
            "The Eiffel Tower is located in Paris, France.",
        ]
    }
)

prompt = Prompt.from_string("Extract named entities from the following text: {{text}}")
task = Task(prompt=prompt, response=NamedEntities)
result = await task(context, model="openai/gpt-3.5-turbo", n_concurrent=10)
print(result.to_pandas(explode=False))

In [ ]:
print(result.to_pandas(explode=True))

# Prompts

In [ ]:
from cuery import Prompt

prompt = Prompt.from_string("Explain {{ topic }} in simple terms.")
c.print(prompt)

# Context

In [ ]:
from cuery.context import iter_context

df = pd.DataFrame(
    {
        "topic": ["Machine Learning", "Natural Language Processing", "Computer Vision"],
        "audience": ["beginners", "developers", "researchers"],
    }
)

contexts, count = iter_context(df, required=["topic", "audience"])
next(contexts)

# Response, Task

In [ ]:
from cuery import Field, Prompt, Response, Task


class MovieRecommendation(Response):
    title: str
    year: int = Field(gt=1900, lt=2030)
    genre: list[str]
    rating: float = Field(ge=0, le=10)


# Multi-output response model
class MovieRecommendations(Response):
    recommendations: list[MovieRecommendation]


movie_prompt = Prompt.from_string(
    "Recommend a movie for {{ audience }} interested in {{ topic }}."
)
context = [
    {"topic": "Machine Learning", "audience": "beginners"},
    {"topic": "Computer Vision", "audience": "researchers"},
]

movie_task = Task(prompt=movie_prompt, response=MovieRecommendations)
result = await movie_task(context)
print(result)
print(result.to_pandas())

In [ ]:
print(result.usage())

In [ ]:
from cuery import pprint

print(result[0]._raw_response.model)
print(result[0]._raw_response.service_tier)

In [ ]:
print(movie_task.queries[0]["messages"][0]["content"])

In [ ]:
c.print(result[0])

# Chain

In [ ]:
from typing import Literal

from cuery import Chain, Task

# Reuse example from above
movies = Task(prompt=movie_prompt, response=MovieRecommendations)


# Add PG rating
class Rating(Response):
    pg_category: Literal["G", "PG", "PG-13", "R", "NC-17"]
    """PG rating of the movie."""
    pg_reason: str
    """Reason for the rating."""


rating_prompt = Prompt.from_string("What is the PG rating for {{ title }}?")
rating_task = Task(prompt=rating_prompt, response=Rating)

# Create a chain of tasks
chain = Chain(movie_task, rating_task)
result = await chain(context, model="openai/gpt-3.5-turbo", n_concurrent=20)
result

In [ ]:
print(result)

# Templates

In [ ]:
from cuery import templates
from cuery.utils import render_template

prompt = """
Analyze the following record and provide a summary in Markdown format.
Make sure to include the name, age, city, and interests.
"""

context = {
    "record": {
        "name": "John Doe",
        "age": 30,
        "city": "New York",
        "interests": ["reading", "traveling", "coding"],
    }
}


In [ ]:
for mode in ["json", "text", "md"]:
    tpl = templates.prompt_with_template(prompt, template=f"record_to_{mode}")
    print(render_template(tpl, **context), end="\n\n")